In [ ]:
import gymnasium as gym
import numpy as np

In [ ]:
episodes = 10


In [ ]:
env = gym.make("Blackjack-v1", render_mode='human')

state_size = len(env.observation_space)
action_size = env.action_space.n


In [ ]:
for e in range(1, episodes+1):
    terminated = False
    state,_ = env.reset()

    while(not terminated):
        action = env.action_space.sample()
        next_state, reward, terminated,_,_ = env.step(action)
        state = next_state

    env.close()